In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import dask.dataframe as dd
from typing import List, Dict
from dask.diagnostics import ProgressBar
ProgressBar().register()

## Test

In [2]:
# replace_dict = {'bank corporation': 'bank', 'bank corporate': 'bank', 'bank corp': 'bank'}

# test_names = ['ORIX Bank Corporation', 'LLOYDS BANK CORPORATE MARKETS PLC', 'Trustco Bank Corp']

In [3]:
# from typing import List, Dict

# to_remove_words_list = [
#     "inc",
#     "inc.",
#     "incorporated",
#     "llc",
#     "l.l.c.",
#     "l.l.c",
#     "co.",
#     "securities",
#     "security",
#     "s.a.",
#     "s.a",
#     "l.p.",
#     "plc",
#     "ag",
#     "s.a.s",
#     "s.a.s.",
#     "s.p.a.",
#     "s.p.a",
#     "sa",
#     "ab",
#     "trust",
#     "limited",
#     "ltd",
#     "ltd.",
# ]

# replace_dict = {'bank corporation': 'bank', 'bank corporate': 'bank', 'bank corp': 'bank'}

# def preprocess(cur_txt: str, to_remove_words_list: List=to_remove_words_list, replace_dict: Dict=replace_dict) -> str:
#     """
#     Preprocess names before matching:
#     1. remove commas
#     2. replace by name
#     3. remove words in to_remove_words_list

#     Args:
#         cur_txt (str): current name
#         to_remove_words_list (List[srt]): list of words to remove

#     Returns:
#         str: preprocessed equity name
#     """
#     #* 0. lower the name
#     cur_txt = cur_txt.lower()
#     #* 1. replace comma
#     cur_txt = cur_txt.replace(",", "")
#     #* 2. replace by name
#     for cur_key in replace_dict:
#         if cur_key in cur_txt:
#             cur_txt = cur_txt.replace(cur_key, replace_dict[cur_key])
#             break
#     #* 3. remove words in to_remove_words_list
#     cur_txt = cur_txt.split(" ")
#     result_str_list = [
#         cur_word
#         for cur_word in cur_txt
#         if cur_word not in to_remove_words_list
#     ]

#     return " ".join(result_str_list).strip()

In [4]:
# preprocess(test_names[2])

## Convert to csv

In [5]:
# search df
search_df = pd.read_excel(os.path.join('data', '01_raw', 'search_df.xlsx'))
search_df.head()

,Full Name,No.
0,1 MAJ Metalska,1
1,1 MAJ Zavrsni Rad Gradjevinarstvu,2
2,1300 Smiles,3
3,2 Park Street Trust,4
4,25 Novembar Celinac,5


In [6]:
search_df.to_csv(os.path.join('data', '02_primary', 'search.csv'), index=False)

In [7]:
# target df
target_df = pd.read_excel(os.path.join('data', '01_raw', 'target_df.xlsx'))
target_df.head()

,Target Name,Target No.
0,360 Capital Industrial Fund,3543
1,ABPA HOLDINGS LIMITED,9511
2,"AIG International, Inc.",21931
3,ALD International S.A.,8556
4,AmBev International Finance Co. Ltd.,4155


In [8]:
target_df.to_csv(os.path.join('data', '02_primary', 'target.csv'), index=False)

In [9]:
target_df.shape

(588, 2)

In [10]:
target_df.columns

Index(['Target Name', 'Target No.'], dtype='object')

## Split Target Df


In [11]:
target_df = pd.read_csv("data/02_primary/target.csv")
print(target_df.shape)
target_df = dd.from_pandas(target_df, npartitions=30)
target_df

(588, 2)


,Target Name,Target No.
npartitions=30,,
0,object,int64
20,...,...
...,...,...
580,...,...
587,...,...


In [12]:
target_df.to_csv('data/04_splits/*.csv', index=False)

[########################################] | 100% Completed |  0.1s


['/research/haohang/text_matching/data/04_splits/00.csv',
 '/research/haohang/text_matching/data/04_splits/01.csv',
 '/research/haohang/text_matching/data/04_splits/02.csv',
 '/research/haohang/text_matching/data/04_splits/03.csv',
 '/research/haohang/text_matching/data/04_splits/04.csv',
 '/research/haohang/text_matching/data/04_splits/05.csv',
 '/research/haohang/text_matching/data/04_splits/06.csv',
 '/research/haohang/text_matching/data/04_splits/07.csv',
 '/research/haohang/text_matching/data/04_splits/08.csv',
 '/research/haohang/text_matching/data/04_splits/09.csv',
 '/research/haohang/text_matching/data/04_splits/10.csv',
 '/research/haohang/text_matching/data/04_splits/11.csv',
 '/research/haohang/text_matching/data/04_splits/12.csv',
 '/research/haohang/text_matching/data/04_splits/13.csv',
 '/research/haohang/text_matching/data/04_splits/14.csv',
 '/research/haohang/text_matching/data/04_splits/15.csv',
 '/research/haohang/text_matching/data/04_splits/16.csv',
 '/research/ha

## Preprocess Search df

In [13]:
search_df = pd.read_csv(os.path.join('data', '02_primary', 'search.csv'))
search_df.shape

(317751, 2)

In [14]:
to_remove_words_list = [
    "inc",
    "inc.",
    "incorporated",
    "llc",
    "l.l.c.",
    "l.l.c",
    "co.",
    "securities",
    "security",
    "s.a.",
    "s.a",
    "l.p.",
    "plc",
    "ag",
    "s.a.s",
    "s.a.s.",
    "s.p.a.",
    "s.p.a",
    "sa",
    "ab",
    "trust",
    "limited",
    "ltd",
    "ltd.",
]

replace_dict = {'bank corporation': 'bank', 'bank corporate': 'bank', 'bank corp': 'bank'}

In [15]:
def preprocess(cur_txt: str, to_remove_words_list: List=to_remove_words_list, replace_dict: Dict=replace_dict) -> str:
    """
    Preprocess names before matching:
    1. remove commas
    2. replace by name
    3. remove words in to_remove_words_list

    Args:
        cur_txt (str): current name
        to_remove_words_list (List[srt]): list of words to remove

    Returns:
        str: preprocessed equity name
    """
    #* 0. lower the name
    cur_txt = cur_txt.lower()
    #* 1. replace comma
    cur_txt = cur_txt.replace(",", "")
    #* 2. replace by name
    for cur_key in replace_dict:
        if cur_key in cur_txt:
            cur_txt = cur_txt.replace(cur_key, replace_dict[cur_key])
            break
    #* 3. remove words in to_remove_words_list
    cur_txt = cur_txt.split(" ")
    result_str_list = [
        cur_word
        for cur_word in cur_txt
        if cur_word not in to_remove_words_list
    ]

    return " ".join(result_str_list).strip()

In [16]:
search_df['preprocessed'] = search_df['Full Name'].apply(preprocess)
with open(os.path.join('data', '03_feature', 'search_df.pkl'), 'wb') as f:
    pickle.dump(search_df, f)

In [17]:
# search_df['preprocessed'] = search_df['Full Name'].apply(preprocess)
# search_list = search_df['preprocessed'].tolist()

In [18]:
# search_df_preprocessed2full_name = {cur_row['preprocessed']: cur_row['Full Name'] for _, cur_row in search_df.iterrows()}
# search_df_full_name2id = {cur_row['Full Name']: cur_row['No. '] for _, cur_row in search_df.iterrows()}

In [ ]:
# # save as pickle
# with open(os.path.join('data', '03_feature', 'search_list.pkl'), 'wb') as f:
#     pickle.dump(search_list, f)
# with open(os.path.join('data', '03_feature', 'search_df_preprocessed2full_name.pkl'), 'wb') as f:
#     pickle.dump(search_df_preprocessed2full_name, f)
# with open(os.path.join('data', '03_feature', 'search_df_full_name2id.pkl'), 'wb') as f:
#     pickle.dump(search_df_full_name2id, f)